In [1]:
# stitch_daily_periods_vertical.py
from pathlib import Path
import re
from PIL import Image
import config

TICKER = getattr(config, "STOCK_TICKER", "VOO")
FIG_DIR = Path(getattr(config, "FIGURE_PATH", "./stock_image/"))

# Match files like: VOO_daily_close_30.png
PATTERN = re.compile(rf"^{re.escape(TICKER)}_(daily_[a-z0-9_]+)_(\d+)\.png$", re.IGNORECASE)

def discover_daily_images():
    """Return { base -> { period:int -> Path } }."""
    groups = {}
    for p in FIG_DIR.glob("*.png"):
        m = PATTERN.match(p.name)
        if not m:
            continue
        base = m.group(1).lower()
        period = int(m.group(2))
        groups.setdefault(base, {})[period] = p
    return groups

def stitch_vertical(paths):
    """Stack images vertically; resize to same width for alignment."""
    imgs = [Image.open(p) for p in paths]
    min_w = min(im.width for im in imgs)
    resized = []
    for im in imgs:
        if im.width != min_w:
            new_h = int(im.height * (min_w / im.width))
            im = im.resize((min_w, new_h), Image.BICUBIC)
        resized.append(im)
    total_h = sum(im.height for im in resized)
    out = Image.new("RGB", (min_w, total_h), "white")
    y = 0
    for im in resized:
        out.paste(im, (0, y))
        y += im.height
    return out

def main():
    FIG_DIR.mkdir(parents=True, exist_ok=True)
    groups = discover_daily_images()
    if not groups:
        print("No matching daily images found.")
        return

    for base, period_map in groups.items():
        periods = sorted(period_map.keys())
        if len(periods) < 2:
            continue  # skip if only one period exists
        paths = [period_map[p] for p in periods]
        combined = stitch_vertical(paths)
        suffix = "-".join(str(p) for p in periods)
        out_name = f"{TICKER}_{base}_{suffix}.png"
        save_path = Path(config.REPORT_PATH)
        save_path.mkdir(parents=True, exist_ok=True)
        out_path = save_path / out_name
        combined.save(out_path)
        print(f"✔ Saved {out_path}")

if __name__ == "__main__":
    main()

✔ Saved report/VOO/VOO_daily_atr_30-60-90.png
✔ Saved report/VOO/VOO_daily_close_30-60-90.png


✔ Saved report/VOO/VOO_daily_rvol_30-60-90.png
✔ Saved report/VOO/VOO_daily_volume_30-60-90.png
